In [9]:
import pandas as pd
import datetime as dt
import math
import statistics as stat
import os

In [10]:
path = "/Users/levimunster/Desktop/VS_Code/MiddleburyCS/stat0350/steam_db_playerdata"
out_path = "/Users/levimunster/Desktop/VS_Code/MiddleburyCS/stat0350/survival_output"

In [11]:
games_list = os.listdir(path)

In [12]:
'.DS_Store' in games_list

True

In [13]:
output_df = pd.DataFrame(columns = ['Game'])

In [14]:
nov_list = []
nov_counter = 0
counter = 0
for i, game_csv in enumerate(games_list):
    if game_csv[0] != ".":
        game = game_csv[:-4]
        df = pd.read_csv(f'{path}/{game}.csv')
        df['DateTime'] = pd.to_datetime(df['DateTime']).dt.normalize()
        daily_df = df.groupby(df['DateTime'].dt.date).agg({
        'Average Players': 'mean'
    }).reset_index()
        daily_df.head(10)
        daily_df = daily_df.dropna(how='any')
        counter += 1
        start_date = daily_df['DateTime'].iloc[0]
        if str(start_date)[0:4] == '2025':
            nov_list.append(game)
            nov_counter += 1



In [ ]:
retention_df = pd.DataFrame(columns=['game', 'peak_n', 'peak_date', 'D1_r', 'D7_r', 'D30_r', 'D360_r', 'D720_r','D1_n', 'D7_n', 'D30_n', 'D360_n', 'D720_n',])

In [16]:
skip_list = []
n_list = [1, 7, 30, 360, 720]
for i, game_csv in enumerate(games_list):
    game = game_csv[:-4]
    if game[0] != '.':
        df = pd.read_csv(f'{path}/{game}.csv')
        df['DateTime'] = pd.to_datetime(df['DateTime']).dt.normalize()
        daily_df = df.groupby(df['DateTime'].dt.date).agg({
        'Players': 'mean'
    }).reset_index()
        daily_df = daily_df.dropna(how='any')
        peak_row = daily_df[daily_df['Players'] == max(daily_df['Players'])]
        peak_date = daily_df.loc[daily_df['Players'].idxmax(), 'DateTime']
        peak_players = float(peak_row['Players'])
        
        row_to_add = [game, peak_players, peak_date]
        for n in n_list:
            try:
                row = daily_df[daily_df['DateTime'] == peak_date + dt.timedelta(days=n)]
                Dn_n = float(daily_df[daily_df['DateTime'] == peak_date + dt.timedelta(days=n)]['Players'])
                Dn_r = (Dn_n / peak_players)
                row_to_add.append(Dn_r)
            except TypeError:
                skip_list.append(game)
                row_to_add.append(float('nan'))
        for n in n_list:
            try:
                row = daily_df[daily_df['DateTime'] == peak_date + dt.timedelta(days=n)]
                Dn_n = float(daily_df[daily_df['DateTime'] == peak_date + dt.timedelta(days=n)]['Players'])
                row_to_add.append(Dn_n)
            except TypeError:
                row_to_add.append(float('nan'))
                pass
        retention_df.loc[len(retention_df)] = row_to_add

In [17]:
retention_df.to_csv('clean_retention2.csv', index=False)